In [1]:
!pip install memory_profiler

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for memory_profiler from https://files.pythonhosted.org/packages/49/26/aaca612a0634ceede20682e692a6c55e35a94c21ba36b807cc40fe910ae1/memory_profiler-0.61.0-py3-none-any.whl.metadata

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import os
import h5py
import numpy as np
import glob
import torch
import sys
import xarray as xr
from netCDF4 import Dataset as ncDataset
from collections import defaultdict

def get_sorted_file_paths(base_path='/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0'):
    h5_files = glob.glob(os.path.join(base_path, '**', '*.h5'), recursive=True)
    return sorted(h5_files)

def _slice_hdf5(path, channel):
    with h5py.File(path, 'r') as h5_file:
        _data = np.array(h5_file["fields"][:, channel, :, :], dtype=np.float64)
        return _data

def _slice_hdf5_new(path, channel):
    with h5py.File(path, 'r') as h5_file:
        _data = np.array(h5_file["fields"][:, channel, :, :], dtype=np.float64)
        if channel == 20:
            _data[_data == -32767.0] = np.nan
        return _data

import psutil
def get_memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / (1024 * 1024)  # Memory in MiB
#%load_ext memory_profiler
get_memory_usage()

525.1875

In [4]:
time_diff_std_new_a = np.load("/iopsstor/scratch/cscs/stefschu/DSM500/gitlab/stats/outputs/graphcast/time_diff_std_new.npy")
time_diff_std_new_b = np.load("/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/stats/time_diff_std_new.npy")

np.allclose(time_diff_std_new_a, time_diff_std_new_b)

True

In [5]:
time_diff_mean_new_a = np.load("/iopsstor/scratch/cscs/stefschu/DSM500/gitlab/stats/outputs/graphcast/time_diff_mean_new.npy")
time_diff_mean_new_b = np.load("/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/stats/time_diff_mean_new.npy")

np.allclose(time_diff_mean_new_a, time_diff_mean_new_b)

True

In [9]:
%%memit
_data_without = _slice_hdf5("/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train/1990.h5", 20)

peak memory: 17780.94 MiB, increment: 17247.88 MiB


In [2]:
%%memit
_data_with = _slice_hdf5_new("/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train/1990.h5", 20)

UsageError: Cell magic `%%memit` not found.


In [19]:
_data = _slice_hdf5_new("/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train/1990.h5", 20)
_data.shape

(1460, 721, 1440)

In [3]:
means_with_sst_fix = np.load("/iopsstor/scratch/cscs/stefschu/DSM500/gitlab/stats/global_means_with_sst_fix.npy")
stds_with_sst_fix = np.load("/iopsstor/scratch/cscs/stefschu/DSM500/gitlab/stats/global_stds_with_sst_fix.npy")

means_with_sst_fix.shape, stds_with_sst_fix.shape

((1, 21, 1, 1), (1, 21, 1, 1))

In [4]:
np.save("/iopsstor/scratch/cscs/stefschu/DSM500/gitlab/stats/global_means_with_sst_fix.npy", means_with_sst_fix.astype(np.float32))
np.save("/iopsstor/scratch/cscs/stefschu/DSM500/gitlab/stats/global_stds_with_sst_fix.npy", stds_with_sst_fix.astype(np.float32))

In [14]:
_data.dtype, means_with_sst_fix.dtype, stds_with_sst_fix.dtype

(dtype('float64'), dtype('float128'), dtype('float128'))

In [15]:
np.sum(~np.isnan(_data))

1002091440

In [16]:
_data[:, :, :] = 1.
_data.shape

(1460, 721, 1440)

In [17]:
np.sum(~np.isnan(_data))

1515830400

In [23]:
np.nanstd(_data)

11.50638266495773

In [24]:
stds_with_sst_fix[0, 20]

array([[11.55724928]], dtype=float128)

In [13]:
%%memit
np.mean(_data_without, dtype=np.float128)

peak memory: 23662.88 MiB, increment: 0.00 MiB


In [14]:
%%memit
np.nanmean(_data_with, dtype=np.float128)

peak memory: 38117.19 MiB, increment: 14454.31 MiB


In [15]:
%%memit
np.sum(~np.isnan(_data_with), dtype=np.float128)

peak memory: 25108.69 MiB, increment: 1445.19 MiB


In [3]:
rank = 10

# Translate parameters
file_path_id = rank // 21
file_path = get_sorted_file_paths()[file_path_id]
channel_id = rank % 21
print(f"Rank {rank:03} processing channel {channel_id:02} of {file_path_id=} {file_path}")

Rank 010 processing channel 10 of file_path_id=0 /iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/out_of_sample/2018.h5


In [9]:
global_means = np.load(f"/iopsstor/scratch/cscs/stefschu/DSM500/gitlab/stats/global_means.npy")
channel_mean = global_means.squeeze()[channel_id]
local_channel_data = _slice_hdf5(file_path, channel_id)

channel_mean, channel_mean.dtype, local_channel_data.dtype

(0.14251510029185673256096374097408843, dtype('float128'), dtype('float64'))

In [10]:
get_memory_usage()

12096.5625

In [16]:
%%memit
local_channel_data = local_channel_data - channel_mean

peak memory: 22545.12 MiB, increment: 10443.38 MiB


In [20]:
%%memit
local_parts = list()
local_parts.append(np.sum(np.square(local_channel_data[1200:] - channel_mean), dtype=np.float128))

peak memory: 15586.69 MiB, increment: 3484.75 MiB


In [ ]:
directory_path = '/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0'
h5_files = glob.glob(os.path.join(directory_path, '**', '*.h5'), recursive=True)
h5_files = sorted(h5_files)
h5_files

In [3]:
means_nersc = np.load("/iopsstor/scratch/cscs/stefschu/DSM500/stats/global_means_nersc.npy")
stds_nersc = np.load("/iopsstor/scratch/cscs/stefschu/DSM500/stats/global_stds_nersc.npy")

means_nersc.shape, stds_nersc.shape

((1, 21, 1, 1), (1, 21, 1, 1))

In [5]:
means_no_sst_fix = np.load("/iopsstor/scratch/cscs/stefschu/DSM500/gitlab/stats/global_means_no_sst_fix.npy")
stds_no_sst_fix = np.load("/iopsstor/scratch/cscs/stefschu/DSM500/gitlab/stats/global_stds_no_sst_fix.npy")

means_no_sst_fix.shape, stds_no_sst_fix.shape

((1, 21, 1, 1), (1, 21, 1, 1))

In [7]:
means_with_sst_fix = np.load("/iopsstor/scratch/cscs/stefschu/DSM500/gitlab/stats/global_means_with_sst_fix.npy")
stds_with_sst_fix = np.load("/iopsstor/scratch/cscs/stefschu/DSM500/gitlab/stats/global_stds_with_sst_fix.npy")

means_with_sst_fix.shape, stds_with_sst_fix.shape

((1, 21, 1, 1), (1, 21, 1, 1))

In [9]:
means_nersc = means_nersc.squeeze()
means_no_sst_fix = means_no_sst_fix.squeeze()
means_with_sst_fix = means_with_sst_fix.squeeze()

stds_nersc = stds_nersc.squeeze()
stds_no_sst_fix = stds_no_sst_fix.squeeze()
stds_with_sst_fix = stds_with_sst_fix.squeeze()

In [34]:
for c in range(21):
    diff = np.abs(means_no_sst_fix[c]-means_nersc[c])
    print(f"Channel {c:>2}: {means_no_sst_fix[c]:25.15f} {means_nersc[c]:25.15f} ({diff:9.5f})  ({np.abs(diff*100/means_no_sst_fix[c]):10.5f}%)")

Channel  0:        -0.048774521750804        -0.006421389803290 (  0.04235)  (  86.83454%)
Channel  1:         0.189605237884631         0.293685093522072 (  0.10408)  (  54.89292%)
Channel  2:       278.453116238492555       279.020666503906227 (  0.56755)  (   0.20382%)
Channel  3:     96650.386689044637023     96601.067187499997090 ( 49.31950)  (   0.05103%)
Channel  4:    100957.487968644374632    100906.415624999994179 ( 51.07234)  (   0.05059%)
Channel  5:       274.528389576391021       274.948004150390602 (  0.41961)  (   0.15285%)
Channel  6:        -0.030998064141286         0.010713329911232 (  0.04171)  ( 134.56129%)
Channel  7:         0.187103207878097         0.300078022480011 (  0.11297)  (  60.38101%)
Channel  8:       737.073849346994166       698.255676269531250 ( 38.81817)  (   5.26652%)
Channel  9:         1.417158680281130         1.477350687980652 (  0.06019)  (   4.24737%)
Channel 10:         0.142515100291857         0.215618632733822 (  0.07310)  (  51.29529%)

In [36]:
for c in range(21):
    diff = np.abs(stds_no_sst_fix[c]-stds_nersc[c])
    print(f"Channel {c:>2}: {stds_no_sst_fix[c]:25.15f} {stds_nersc[c]:25.15f} ({diff:12.5f})  ({diff*100/stds_no_sst_fix[c]:10.5f}%)")

Channel  0:         5.543580340199223         5.612823667305403 (     0.06924)  (   1.24907%)
Channel  1:         4.762262300438604         4.779908035542846 (     0.01765)  (   0.37053%)
Channel  2:        21.289407924965573        21.424178310025468 (     0.13477)  (   0.63304%)
Channel  3:      9587.904724292517130      9669.023197820966743 (    81.11847)  (   0.84605%)
Channel  4:      1332.690082683000355      1386.300842169548787 (    53.61076)  (   4.02275%)
Channel  5:        15.625534016319731        15.740415145265784 (     0.11488)  (   0.73521%)
Channel  6:         6.137201304841411         6.196485803244808 (     0.05928)  (   0.96599%)
Channel  7:         5.304173277914519         5.309539149341830 (     0.00537)  (   0.10116%)
Channel  8:      1072.826864554260510      1129.480599656320010 (    56.65374)  (   5.28079%)
Channel  9:         8.185777989488274         8.281581195470341 (     0.09580)  (   1.17036%)
Channel 10:         6.262028568829255         6.250067992794

In [39]:
for c in range(21):
    diff = np.abs(means_with_sst_fix[c]-means_nersc[c])
    print(f"Channel {c:>2}: {means_with_sst_fix[c]:25.15f} {means_nersc[c]:25.15f} ({diff:12.5f})  ({np.abs(diff*100/means_with_sst_fix[c]):10.5f}%)")

Channel  0:        -0.048774521750804        -0.006421389803290 (     0.04235)  (  86.83454%)
Channel  1:         0.189605237884631         0.293685093522072 (     0.10408)  (  54.89292%)
Channel  2:       278.453116238492555       279.020666503906227 (     0.56755)  (   0.20382%)
Channel  3:     96650.386689044637023     96601.067187499997090 (    49.31950)  (   0.05103%)
Channel  4:    100957.487968644374632    100906.415624999994179 (    51.07234)  (   0.05059%)
Channel  5:       274.528389576391021       274.948004150390602 (     0.41961)  (   0.15285%)
Channel  6:        -0.030998064141286         0.010713329911232 (     0.04171)  ( 134.56129%)
Channel  7:         0.187103207878097         0.300078022480011 (     0.11297)  (  60.38101%)
Channel  8:       737.073849346994166       698.255676269531250 (    38.81817)  (   5.26652%)
Channel  9:         1.417158680281130         1.477350687980652 (     0.06019)  (   4.24737%)
Channel 10:         0.142515100291857         0.215618632733

In [40]:
for c in range(21):
    diff = np.abs(stds_with_sst_fix[c]-stds_nersc[c])
    print(f"Channel {c:>2}: {stds_with_sst_fix[c]:25.15f} {stds_nersc[c]:25.15f} ({diff:12.5f})  ({np.abs(diff*100/stds_with_sst_fix[c]):10.5f}%)")

Channel  0:         5.543580340199223         5.612823667305403 (     0.06924)  (   1.24907%)
Channel  1:         4.762262300438604         4.779908035542846 (     0.01765)  (   0.37053%)
Channel  2:        21.289407924965573        21.424178310025468 (     0.13477)  (   0.63304%)
Channel  3:      9587.904724292517130      9669.023197820966743 (    81.11847)  (   0.84605%)
Channel  4:      1332.690082683000355      1386.300842169548787 (    53.61076)  (   4.02275%)
Channel  5:        15.625534016319731        15.740415145265784 (     0.11488)  (   0.73521%)
Channel  6:         6.137201304841411         6.196485803244808 (     0.05928)  (   0.96599%)
Channel  7:         5.304173277914519         5.309539149341830 (     0.00537)  (   0.10116%)
Channel  8:      1072.826864554260510      1129.480599656320010 (    56.65374)  (   5.28079%)
Channel  9:         8.185777989488274         8.281581195470341 (     0.09580)  (   1.17036%)
Channel 10:         6.262028568829255         6.250067992794

In [21]:
def _slice_hdf5(path, channel):
    with h5py.File(path, 'r') as h5_file:
        _data = np.array(h5_file["fields"][:, channel, :, :], dtype=np.float64)
        return _data

_data = _slice_hdf5("/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train/1990.h5", 20)
_data.shape

(1460, 721, 1440)

In [22]:
np.min(_data)

-32767.0

In [23]:
np.mean(_data), np.std(_data)

(-10915.659749781302, 15645.726426435735)

In [24]:
_data[_data == -32767.0] = np.nan

In [25]:
np.nanmean(_data), np.nanstd(_data)

(286.7958013343402, 11.50638266495773)

In [39]:
np.sum(np.square(np.array([np.nan, 5.]) - np.float64(3.4)))

nan

In [4]:
a = np.array([1, 2, 3], dtype=np.float128)
b = np.array([0.1, 0.2, .3], dtype=np.float128)
a, b

(array([1., 2., 3.], dtype=float128), array([0.1, 0.2, 0.3], dtype=float128))

In [5]:
a/b

array([10., 10., 10.], dtype=float128)